In [7]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import openpyxl
from itertools import combinations
from collections import Counter

## 아이템 상호작용 행렬 만들기

1. 필요한 데이터 로딩

In [4]:
# TODO : 데이터 경로 설정 
raw_data = pd.read_excel("/opt/ml/input/data/raw_codimap/item/view/item.xlsx", engine='openpyxl')
df = raw_data[["id", "codi_id"]]
unique_codi_id = list(df["codi_id"].unique())
unique_id = list(df["id"].unique())
codi_group_by_codi = df.groupby("codi_id")["id"].apply(list)
codi_group_by_item = df.groupby("id")["codi_id"].apply(list)

In [5]:
df["id"].value_counts()

1551840    10
404949      9
2024232     5
1278214     5
1833230     5
           ..
1960520     1
838145      1
399948      1
1647181     1
1937003     1
Name: id, Length: 2381, dtype: int64

In [6]:
cnt = 0
for count in df["id"].value_counts() : 
    if count == 1 :
        cnt +=1

print(f"Number of total instance : {len(df)}")
print(f"Number of unique items : {len(unique_id)}")
print(f"Number of items which was only collected once : {cnt}")

Number of total instance : 2863
Number of unique items : 2381
Number of items which was only collected once : 2038


2. Pivot Table을 위한 데이터 처리 과정

In [14]:
df = pd.read_excel("/opt/ml/input/data/raw_codishop/codi/recent/codi_item_id.xlsx", engine='openpyxl')

dup = list()
gf = df.groupby(['id'])['item_id'].apply(list)
for val in tqdm(gf):
    val = sorted(val)
    comb_res = list(combinations(val, 2))
    dup.extend(comb_res)
Counter(dup).most_common()

100%|██████████| 600/600 [00:00<00:00, 231452.44it/s]


[((2322323, 2322368), 3),
 ((791654, 2460717), 3),
 ((2297365, 2340821), 2),
 ((2106834, 2287021), 2),
 ((2318591, 2341070), 2),
 ((2230659, 2318591), 2),
 ((1871661, 2352253), 2),
 ((2208045, 2298807), 2),
 ((2178537, 2292480), 2),
 ((1523634, 2316780), 2),
 ((2368649, 2372608), 2),
 ((1747276, 2012028), 2),
 ((1432188, 2352234), 2),
 ((1511220, 2380272), 2),
 ((1924983, 2208885), 2),
 ((1388776, 1900410), 2),
 ((1446291, 2267229), 2),
 ((2122443, 2368649), 2),
 ((2239046, 2407735), 2),
 ((385415, 2407486), 2),
 ((2209891, 2439141), 2),
 ((2334069, 2422246), 2),
 ((2396690, 2396697), 2),
 ((1408194, 2281443), 2),
 ((1989374, 2362436), 2),
 ((693598, 2417190), 2),
 ((2224554, 2394639), 2),
 ((1989494, 2412303), 2),
 ((2238671, 2333958), 2),
 ((2383982, 2421338), 2),
 ((2383982, 2439141), 2),
 ((2031713, 2334069), 2),
 ((2295215, 2437101), 2),
 ((2145912, 2424595), 2),
 ((2301968, 2383982), 2),
 ((2301968, 2421338), 2),
 ((2027822, 2104329), 2),
 ((2300783, 2338691), 2),
 ((2316958, 243

3. 상호작용 행렬 만들기 & 데이터 내보내기

In [ ]:
IIM = pd.DataFrame(columns=["id", "item", "count"])
for pair, count in Counter(dup).most_common():
    
IIM.pivot_table(index="id", columns="item", values="count").fillna(0)
    

In [62]:
IIM = pd.DataFrame(result, columns=["id", "item", "count"]).pivot_table(index="id", columns="item", values="count").fillna(0)
# TODO : 데이터 경로 설정
IIM.to_csv("/opt/ml/input/data/itemInteractionMatrix.csv")

4. Sparsity 점검

In [77]:
ones = 0
zeros = 0
for i in range(len(IIM)) :
    zeros += IIM.iloc[i].value_counts()[0]
    ones += IIM.iloc[i].value_counts()[1]
print(f"Sparsity of this Data is : {zeros/(ones+zeros):.4}")

Sparsity of this Data is : 0.997
